In [1]:
%store -r import_all_libs

In [2]:
import_all_libs

In [3]:
from datetime import date
import gc

In [4]:
fat_ljs_atual = pd.read_csv('/home/tuts/datasets/uspvsufmg/faturamento_lojas_atuais.csv')
cen_exp = pd.read_csv('/home/tuts/datasets/uspvsufmg/cenarios_expansao.csv')
ljs_atual = pd.read_csv('/home/tuts/datasets/uspvsufmg/lojas_atuais.csv')

In [5]:
ljs_atual.cod_loja.nunique()

3130

In [6]:
cen_exp.head()

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,False,50.00,FEATURE_05_VALUE_03,6,0.13,0.13,116.19,2.91,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.00,93.40,100.00,100.00,71.70,88.80
1,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,41.00,FEATURE_05_VALUE_03,4,0.12,0.11,26.35,1.31,NaN,NaN,84.30,77.50,100.00,100.00,63.50,78.80
2,16,2928703003003,2928703,NaN,FEATURE_02_VALUE_04,False,42.12,FEATURE_05_VALUE_05,2,0.10,0.09,35.36,1.97,NaN,NaN,nan,nan,nan,nan,nan,nan
3,21,2906006003001,2906006,NaN,FEATURE_02_VALUE_04,False,30.00,FEATURE_05_VALUE_03,6,0.13,0.13,9.86,1.38,NaN,NaN,85.00,76.90,100.00,100.00,100.00,92.90
4,28,3550308005058,3550308,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,True,44.31,FEATURE_05_VALUE_03,8,0.19,0.18,5593.91,2.82,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,nan,nan,nan,nan,nan,nan


In [7]:
df = fat_ljs_atual.merge(ljs_atual, on='cod_loja', how='inner')
df["datetime"] = pd.to_datetime(df.ano*100 + df.mes, format='%Y%m')
df.drop(["ano", "mes"], axis=1, inplace=True)

In [8]:
s = set(fat_ljs_atual.columns) - set(['ano', 'mes'])
s.add('datetime')
s

{'categoria', 'cod_loja', 'datetime', 'qtde', 'receita'}

In [9]:
fat_mes_df = df[list(s)].groupby(["cod_loja", "datetime"] , as_index=False).sum()

In [10]:
df_fat_mes = fat_mes_df.merge(ljs_atual, on="cod_loja", how='inner')

In [11]:
lojas_sem_24_meses = []
for i in df_fat_mes["cod_loja"].unique():
    if df_fat_mes[df_fat_mes["cod_loja"] == i].shape[0] != 24:
        lojas_sem_24_meses.append(i)

train_df = df_fat_mes[~df_fat_mes["cod_loja"].isin(lojas_sem_24_meses).values]

# Imputing

In [12]:
print('## FEATURE 1 ##')
print(train_df.feature_01.describe())
print('## FEATURE 11 ##')
print(train_df.feature_11.describe())
print('## FEATURE 12 ##')
print(train_df.feature_12.describe())
print('## FEATURE 13 ##')
print(train_df.feature_13.describe())
print('## FEATURE 14 ##')
print(train_df.feature_14.describe())
print('## FEATURE 15 ##')
print(train_df.feature_15.describe())
print('## FEATURE 16 ##')
print(train_df.feature_16.describe())
print('## FEATURE 17 ##')
print(train_df.feature_17.describe())
print('## FEATURE 18 ##')
print(train_df.feature_18.describe())

## FEATURE 1 ##
count                   25440
unique                      3
top       FEATURE_01_VALUE_10
freq                    14808
Name: feature_01, dtype: object
## FEATURE 11 ##
count                    5760
unique                      7
top       FEATURE_11_VALUE_02
freq                     2304
Name: feature_11, dtype: object
## FEATURE 12 ##
count                    5760
unique                      2
top       FEATURE_12_VALUE_01
freq                     3600
Name: feature_12, dtype: object
## FEATURE 13 ##
count   20736.00
mean       94.81
std         8.60
min        44.10
25%        92.20
50%       100.00
75%       100.00
max       100.00
Name: feature_13, dtype: float64
## FEATURE 14 ##
count   20736.00
mean       83.70
std        13.58
min         0.00
25%        79.68
50%        86.90
75%        91.90
max       100.00
Name: feature_14, dtype: float64
## FEATURE 15 ##
count   20736.00
mean       82.66
std        34.42
min         0.00
25%       100.00
50%       100.00
75%

In [13]:
has_na_cols = list()
for col in train_df.columns:
    if train_df[col].isnull().values.any():
        has_na_cols.append(col)
has_na_cols

['feature_01',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18']

In [14]:
for na_col in has_na_cols:
    if train_df[na_col].dtype == object:
        print(train_df[na_col].mode()[0])
        train_df[na_col].fillna(train_df[na_col].mode()[0], inplace=True)
    else:
        train_df[na_col].fillna(train_df[na_col].median(), inplace=True)

FEATURE_01_VALUE_10
FEATURE_11_VALUE_02
FEATURE_12_VALUE_01



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
cols = train_df.columns
for col in cols:
    if train_df[col].dtype == object:
        print(col)
        print(train_df[col].value_counts())
        train_df = pd.concat([train_df, pd.get_dummies(train_df[col])], axis=1)

feature_01
FEATURE_01_VALUE_10    60480
FEATURE_01_VALUE_08     5832
FEATURE_01_VALUE_03     4800
Name: feature_01, dtype: int64
feature_02
FEATURE_02_VALUE_06    20400
FEATURE_02_VALUE_03    19608
FEATURE_02_VALUE_02    16272
FEATURE_02_VALUE_04    14832
Name: feature_02, dtype: int64
feature_05
FEATURE_05_VALUE_03    45816
FEATURE_05_VALUE_06    11088
FEATURE_05_VALUE_05    11064
FEATURE_05_VALUE_02     2616
FEATURE_05_VALUE_01      336
FEATURE_05_VALUE_07      144
FEATURE_05_VALUE_08       24
FEATURE_05_VALUE_04       24
Name: feature_05, dtype: int64
feature_11
FEATURE_11_VALUE_02    67656
FEATURE_11_VALUE_01     1416
FEATURE_11_VALUE_06     1104
FEATURE_11_VALUE_07      672
FEATURE_11_VALUE_04       96
FEATURE_11_VALUE_03       96
FEATURE_11_VALUE_05       72
Name: feature_11, dtype: int64
feature_12
FEATURE_12_VALUE_01    68952
FEATURE_12_VALUE_02     2160
Name: feature_12, dtype: int64


In [20]:
train_df.shape

(71112, 48)

In [21]:
model_df = train_df[cols]
model_df["month"] = model_df["datetime"].dt.month
model_df["quarter"] = model_df["datetime"].dt.quarter
model_df["year"] = model_df["datetime"].dt.year


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
str_cols = list()
for col in model_df.columns.values.tolist():
    if model_df[col].dtype == object:
        str_cols.append(col)

In [23]:
str_cols.append('datetime') 
str_cols.append('receita')
str_cols.append('qtde')

In [24]:
str_cols

['feature_01',
 'feature_02',
 'feature_05',
 'feature_11',
 'feature_12',
 'datetime',
 'receita',
 'qtde']

In [25]:
train = model_df[~model_df["datetime"].isin([date(2017,10,1), date(2017,11,1), date(2017,12,1)])]
test = model_df[model_df["datetime"].isin([date(2017,10,1), date(2017,11,1), date(2017,12,1)])]

X_train = train.drop(str_cols, axis=1)
y_train = train["receita"]

X_test = test.drop(str_cols, axis=1)
y_test = test["receita"]

In [26]:
X_train.columns

Index(['cod_loja', 'cod_ap', 'cod_municipio', 'feature_03', 'feature_04',
       'feature_06', 'feature_07', 'feature_08', 'feature_09', 'feature_10',
       'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17',
       'feature_18', 'month', 'quarter', 'year'],
      dtype='object')

# Train

In [27]:
errors = list()

In [ ]:
rf = RandomForestRegressor(n_estimators=400, n_jobs=-1)

rf.fit(X_train.values, y_train.values)
y_pred = rf.predict(X_test)
m = np.sqrt(mean_squared_error(y_test, y_pred))
errors.append(m)
m

In [29]:
lojas = ljs_atual.cod_loja.unique()

In [44]:
len(lojas)

3130

* todas as lojas:
    - valores da loja
    - datas tem que ser 1° trimestre de 2018
    - preve
    - coloca no df

In [26]:
del model_df, train, test, X_train, X_test, y_train, y_test, train_df
gc.collect()

181

# Submission

In [45]:
# copy df_fat_mes
train_df = df_fat_mes

print(train_df.cod_loja.nunique())

# get cols with nans
has_na_cols = list()
for col in train_df.columns:
    if train_df[col].isnull().values.any():
        has_na_cols.append(col)
# input values
for na_col in has_na_cols:
    if train_df[na_col].dtype == object:
        print(train_df[na_col].mode()[0])
        train_df[na_col].fillna(train_df[na_col].mode()[0], inplace=True)
    else:
        train_df[na_col].fillna(train_df[na_col].median(), inplace=True)

# get cols for model
cols = train_df.columns
model_df = train_df[cols]
model_df["month"] = model_df["datetime"].dt.month
model_df["quarter"] = model_df["datetime"].dt.quarter
model_df["year"] = model_df["datetime"].dt.year

# get str and unused cols
str_cols = list()
for col in model_df.columns.values.tolist():
    if model_df[col].dtype == object:
        str_cols.append(col)
str_cols.append('datetime') 
str_cols.append('receita')
str_cols.append('qtde')

# train test split
train = model_df[~model_df["datetime"].isin([date(2017,10,1), date(2017,11,1), date(2017,12,1)])]
test = model_df[model_df["datetime"].isin([date(2017,10,1), date(2017,11,1), date(2017,12,1)])]

print(model_df.cod_loja.nunique())

# X_train = train.drop(str_cols, axis=1)
# y_train = train["receita"]
# X_test = test.drop(str_cols, axis=1)
# y_test = test["receita"]

# predict
train_cols = X_train.columns.values.tolist()
preds = []
i = 0
for loja in lojas:
    Xloja = model_df[model_df.cod_loja == loja][:1]
    i += 1
    Xloja = Xloja[train_cols]
    Xloja = pd.DataFrame(Xloja.values.tolist()*3, columns=Xloja.columns)
    Xloja.month = pd.Series([1,2,3]).values
    Xloja.quarter = pd.Series([1,1,1]).values
    Xloja.year = pd.Series([2018,2018,2018]).values
    yhat = rf.predict(Xloja)
    preds.append((loja, yhat))
print(i)

3130
3130
3130


In [46]:
submission = pd.DataFrame({'cod_loja': [i[0] for i in preds], 'faturamento': [np.sum(i[1]) for i in preds]})
submission.cod_loja.nunique()

3130

In [47]:
submission.to_csv('/home/tuts/repos/others/usp-vs-ufmg/tutss/desafio_1.csv', index=False, sep=';')